# Exploratory Data Analysis 

In this notebook we will analyse the data available to find evidence to support or refute the claim that the Covid19 pandemic affected production in the automotive industry.

## Data sets used

- [OICA - International Organization of Motor Vehicle Manufacturers](https://www.oica.net/production-statistics/)
  - Initially we looked at th OICA data, however it became clear that the data here would not be detailed enough since only yearly summaries of production are available.
- [FRED](https://fred.stlouisfed.org/)
- [EUROSTATS](https://ec.europa.eu/eurostat/en/)
- [COVID-19 Data Repository by the Center for Systems Science and Engineering](https://github.com/CSSEGISandData/COVID-19)


In [1]:
# Imports 
import os
import numpy as np
import pandas as pd
import scipy.stats as st
import researchpy as rp
import pycountry
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Loading data sets 

# Root data dir 
data_dir = os.path.join(os.pardir, 'data')

# Automotive Data
automotive_dir = os.path.join(data_dir, 'automotive')

# OICA Continets data 
oica_continents_pdf = pd.read_csv(os.path.join(automotive_dir, 'oica_continents_17_to_21.csv'))

# OICA Countries data 
oica_countries_pdf = pd.read_csv(os.path.join(automotive_dir, 'oica_countries_17_to_21.csv'))

# FRED DAUPSA
daupsa_pdf = pd.read_csv(os.path.join(automotive_dir, 'FRED_DAUPSA.csv'))

# EU manufacturing 
eu_manufacturing_df = pd.read_csv(os.path.join(automotive_dir, 'sts_inpr_m__custom_1792844_page_linear.csv'))
eu_2018_2020_manufacturing_df = pd.read_csv(os.path.join(automotive_dir, 'sts_inpr_m__custom_1793898_page_linear.csv'))


# Covid data 
covid_year_pdf = pd.read_csv(os.path.join(os.pardir, 'data', 'covid', 'covid_year_agg.csv'))
covid_month_pdf = pd.read_csv(os.path.join(os.pardir, 'data', 'covid', 'covid_month_agg.csv'))
covid_confirmed_pdf = pd.read_csv(os.path.join(os.pardir, 'data', 'covid', 'covid_confirmed.csv'))

# Global new cases
global_new_cases = covid_confirmed_pdf.groupby(['year', 'month']).sum('Confirmed').reset_index().sort_values(by=['year', 'month'])
global_new_cases['New Cases'] = (global_new_cases['Confirmed'] - global_new_cases['Confirmed'].shift(1)).fillna(global_new_cases['Confirmed'])

# US new cases
us_new_cases_pdf = covid_confirmed_pdf[covid_confirmed_pdf['Country_Region']=='US'].sort_values(by=['year', 'month'])
us_new_cases_pdf['New Cases'] = (us_new_cases_pdf['Confirmed'] - us_new_cases_pdf['Confirmed'].shift(1)).fillna(us_new_cases_pdf['Confirmed'])

# China new cases
ch_new_cases_pdf = covid_confirmed_pdf[covid_confirmed_pdf['Country_Region']=='China'].sort_values(by=['year', 'month'])
ch_new_cases_pdf['New Cases'] = (ch_new_cases_pdf['Confirmed'] - ch_new_cases_pdf['Confirmed'].shift(1)).fillna(ch_new_cases_pdf['Confirmed'])

# europe 
country_names = ['Austria', 'Bosnia and Herzegovina', 'Belgium', 'Bulgaria', 'Switzerland', 'Cyprus', 'Czechia', 'Germany', 'Denmark', 'Estonia', 'Greece', 'Spain', 'Finland',
                 'France', 'Croatia', 'Hungary', 'Ireland', 'Italy', 'Lithuania', 'Luxembourg', 'Latvia', 'Montenegro', 'North Macedonia', 'Malta', 'Netherlands', 'Norway', 'Poland',
                 'Portugal', 'Romania', 'Serbia', 'Sweden', 'Slovenia', 'Slovakia', 'Turkey', 'UK']
eu_covid_confirmed = covid_confirmed_pdf[covid_confirmed_pdf.Country_Region.isin(country_names)]
eu_covid_confirmed = eu_covid_confirmed.groupby(['year', 'month']).sum('Confirmed').reset_index()
eu_covid_confirmed['New Cases'] = (eu_covid_confirmed['Confirmed'] - eu_covid_confirmed['Confirmed'].shift(1)).fillna(eu_covid_confirmed['Confirmed'])



## Investigating global trend for automotive production 

For this we will base out figures off the data scrapped form OICA, taking the sum of the continental production values.

In [3]:
# Formatting OICA USA data 2017-2021
usa_oica_pdf =  oica_countries_pdf.melt(id_vars=['country'], var_name='year', value_name='production')
usa_oica_pdf.country = usa_oica_pdf.country.astype(str)
usa_oica_pdf = usa_oica_pdf[usa_oica_pdf['country'] == 'USA']


# Calculating variation and % diff 
usa_oica_pdf['Variation'] = usa_oica_pdf['production'] - usa_oica_pdf['production'].shift(1)
usa_oica_pdf['percentage_diff'] = round(usa_oica_pdf['production']*100/usa_oica_pdf['production'].shift(1)) -100
usa_oica_pdf

# Formatting data 
global_production_pdf = oica_continents_pdf.melt(id_vars=['country'], var_name='year', value_name='production')\
                                           .groupby('year').sum('production').reset_index()

global_production_pdf['month'] = 12
global_production_pdf.loc[global_production_pdf['year'] == '2021', 'month'] = 9
global_production_pdf['day'] = 1
global_production_pdf['DATE'] = pd.to_datetime(global_production_pdf[['year', 'month', 'day']])

# Formatting monthly covid data 
global_new_cases['day'] = 1
global_new_cases['DATE'] = pd.to_datetime(global_new_cases[['year', 'month', 'day']])

# Plotting 

fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=global_new_cases['DATE'], y=global_new_cases['New Cases'], name='Global Covid19 Daily New Cases'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=global_production_pdf['DATE'], y=global_production_pdf['production'], name='Global automotive production'),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title_text="Global OCIA and Covid New Cases"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> Auto production in thousands", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Covid Incident Rate", secondary_y=True)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

From the trend we can see that the global production was already falling prior to 2019. 

We observe a sharper decline between 2019 and 2021 from ~67M to less than 40M. This value however should be only considered as an indicator since there are still statistics missing for 2021, notably from countries that only report data once a year.

We now examine the difference in production a bit closer.

In [4]:
global_production_pdf['Variation'] = global_production_pdf['production'] - global_production_pdf['production'].shift(1)
global_production_pdf['percentage_diff'] = round(global_production_pdf['production']*100/global_production_pdf['production'].shift(1)) -100
global_production_pdf

,year,production,month,day,DATE,Variation,percentage_diff
0,2017,72663012.0,12,1,2017-12-01,NaN,NaN
1,2018,71750946.0,12,1,2018-12-01,-912066.0,-1.0
2,2019,67163769.0,12,1,2019-12-01,-4587177.0,-6.0
3,2020,55834455.0,12,1,2020-12-01,-11329314.0,-17.0
4,2021,39681237.0,9,1,2021-09-01,-16153218.0,-29.0


We see a cumulative loss in production of 7% between 2017 to 2019.

The cumulative loss between 2019 and 2020 is 46%, more than 6 times as much compared to the previous period, while this is not statistical evidence that the pandemic is directly responsible it is clear to see that an outside factor has cause production to decline.

## USA automotive production

Limited by the available data we will now focus on automotive production in the USA.

In [4]:
# Formatting OICA USA data 2017-2021
usa_oica_pdf =  oica_countries_pdf.melt(id_vars=['country'], var_name='year', value_name='production')
usa_oica_pdf.country = usa_oica_pdf.country.astype(str)
usa_oica_pdf = usa_oica_pdf[usa_oica_pdf['country'] == 'USA']
usa_oica_pdf['month'] = 12
usa_oica_pdf.loc[usa_oica_pdf['year'] == '2021', 'month'] = 9
usa_oica_pdf['day'] = 1
usa_oica_pdf['DATE'] = pd.to_datetime(usa_oica_pdf[['year', 'month', 'day']])
# filtering out values 
usa_oica_pdf = usa_oica_pdf[usa_oica_pdf['DATE']>'2019-11']

# Calculating variation and % diff 
usa_oica_pdf['Variation'] = usa_oica_pdf['production'] - usa_oica_pdf['production'].shift(1)
usa_oica_pdf['percentage_diff'] = round(usa_oica_pdf['production']*100/usa_oica_pdf['production'].shift(1)) -100


# Formatting monthly covid data 
us_new_cases_pdf['day'] = 1
us_new_cases_pdf['DATE'] = pd.to_datetime(us_new_cases_pdf[['year', 'month', 'day']])

# Plotting 

fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=us_new_cases_pdf['DATE'], y=us_new_cases_pdf['New Cases'], name='US Covid19 Daily New Cases'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=usa_oica_pdf['DATE'], y=usa_oica_pdf['production'], name='US automotive production'),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title_text="US OCIA and Covid New Cases"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> Auto production in thousands", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Covid Incident Rate", secondary_y=True)
fig.add_vrect(x0='2020-03-01', x1='2020-04-07', line_width=0, fillcolor="grey", opacity=0.4,
                annotation_text="lockdown", annotation_position="top left",)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

The OICA data limits us to yearly observations so now we will look at the FRED Domestic Auto Production (DAUPSA). Domestic auto production is defined as all autos assembled in the U.S.

In [16]:
# formatting daupsa data 
daupsa_pdf.DATE = pd.to_datetime(daupsa_pdf.DATE) 
# filtering out values 
filtered_daupsa_pdf = daupsa_pdf[daupsa_pdf['DATE']>'2019-11']


# Formatting monthly covid data 
us_new_cases_pdf['day'] = 1
us_new_cases_pdf['DATE'] = pd.to_datetime(us_new_cases_pdf[['year', 'month', 'day']])

# Plotting 
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=us_new_cases_pdf['DATE'], y=us_new_cases_pdf['New Cases'], name='US Covid19 Daily New Cases'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=filtered_daupsa_pdf['DATE'], y=filtered_daupsa_pdf['DAUPSA'], name='Auto units produced'),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title_text="DAUPSA and Covid Incident Rate"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> Auto production in thousands", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Covid Incident Rate", secondary_y=True)
fig.add_vrect(x0='2020-03-01', x1='2020-04-07', line_width=0, fillcolor="grey", opacity=0.4,
                annotation_text="lockdown", annotation_position="top left",)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()


### T test 

In [11]:
summary, results = rp.ttest(group1= daupsa_pdf['DAUPSA'][daupsa_pdf.DATE<'2020'], group1_name= "pre-covid",
                            group2= daupsa_pdf['DAUPSA'][daupsa_pdf.DATE>='2020'], group2_name= "covid")

In [12]:
print(summary)

    Variable     N        Mean         SD         SE   95% Conf.    Interval
0  pre-covid  24.0  220.429167  18.862004   3.850190  212.464441  228.393892
1      covid  22.0  146.818182  55.039705  11.734505  122.414944  171.221420
2   combined  46.0  185.223913  54.568087   8.045625  169.019192  201.428634


In [13]:
print(results)

                  Independent t-test  results
0  Difference (pre-covid - covid) =   73.6110
1              Degrees of freedom =   44.0000
2                               t =    6.1737
3           Two side test p value =    0.0000
4          Difference < 0 p value =    1.0000
5          Difference > 0 p value =    0.0000
6                       Cohen's d =    1.8223
7                       Hedge's g =    1.7910
8                   Glass's delta =    3.9026
9                     Pearson's r =    0.6813


We can see the difference mean production post/during covid is significantly less than with a confidence of 95%, therefore we can reject the null hypothesis that the the means remained the same.

## EU & Europe Manufacturing Data 

In [17]:
# formatting EUROSTATS data 
eu_manufacturing_df = eu_manufacturing_df[['geo', 'TIME_PERIOD','OBS_VALUE']]
eu_manufacturing_df['DATE'] = pd.to_datetime(eu_manufacturing_df['TIME_PERIOD'])
europe_manufacturing_pdf = eu_manufacturing_df[~eu_manufacturing_df.geo.isin(['EA19', 'EA27', 'EU27_2020', 'US'])]
europe_manufacturing_pdf = europe_manufacturing_pdf.groupby('DATE').mean('OBS_VALUE').reset_index()

# Formatting monthly covid data 
eu_covid_confirmed['day'] = 1
eu_covid_confirmed['DATE'] = pd.to_datetime(eu_covid_confirmed[['year', 'month', 'day']])

# Plotting 
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=eu_covid_confirmed['DATE'], y=eu_covid_confirmed['New Cases'], name='Covid New Cases (EUROPE)'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=europe_manufacturing_pdf['DATE'], y=europe_manufacturing_pdf['OBS_VALUE'], name='EUROSTATS Volume index'),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title_text="EUROSTATS Volume index of production and Covid New Cases (EUROPE)"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> percentage change from previous period", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Covid New Cases (EUROPE)", secondary_y=True)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

### T test 

In [26]:
eu_2018_2020_manufacturing_df
eu_2018_2020_manufacturing_df = eu_2018_2020_manufacturing_df[['geo', 'TIME_PERIOD','OBS_VALUE']]
eu_2018_2020_manufacturing_df['DATE'] = pd.to_datetime(eu_2018_2020_manufacturing_df['TIME_PERIOD'])
eu_2018_2020_manufacturing_df = eu_2018_2020_manufacturing_df[~eu_2018_2020_manufacturing_df.geo.isin(['EA19', 'EA27', 'EU27_2020', 'US'])]
eu_2018_2020_manufacturing_df = eu_2018_2020_manufacturing_df.groupby('DATE').mean('OBS_VALUE').reset_index()


In [27]:
summary, results = rp.ttest(group1= eu_2018_2020_manufacturing_df['OBS_VALUE'][eu_2018_2020_manufacturing_df.DATE<'2020'], group1_name= "pre-covid",
                            group2= eu_2018_2020_manufacturing_df['OBS_VALUE'][eu_2018_2020_manufacturing_df.DATE>='2020'], group2_name= "covid")

In [28]:
print(summary)

    Variable     N      Mean        SD        SE  95% Conf.  Interval
0  pre-covid  24.0  0.030000  0.652617  0.133215  -0.245576  0.305576
1      covid  22.0  0.387843  4.879363  1.040284  -1.775545  2.551231
2   combined  46.0  0.201142  3.370584  0.496965  -0.799797  1.202082


In [29]:
print(results)

                  Independent t-test  results
0  Difference (pre-covid - covid) =   -0.3578
1              Degrees of freedom =   44.0000
2                               t =   -0.3562
3           Two side test p value =    0.7234
4          Difference < 0 p value =    0.3617
5          Difference > 0 p value =    0.6383
6                       Cohen's d =   -0.1051
7                       Hedge's g =   -0.1033
8                   Glass's delta =   -0.5483
9                     Pearson's r =    0.0536
